# Setup

* Importing libraries required for programs

* Setting up variables for file paths, field datatypes , keys and other important items that would be used later

**Sources**
1. GDELT Schema information: http://data.gdeltproject.org/documentation/GDELT-Event_Codebook-V2.0.pdf

2. About GDELT duplicate issue: http://causalloop.blogspot.com/2014/05/how-bad-are-duplication-problems-in.html


### Task

**Description:**

GDELT is a large public dataset provided by Google. 

It basically consists of three parts: Events, Mentions and the Global Knowledge Graph (GKG). The full dataset is in the terabytes and updates every 15 minutes. 

For now, we just care about the Events dataset.

 

Google releases the data every 15 minutes as zipped CSV files which can be found here:

http://data.gdeltproject.org/gdeltv2/lastupdate.txt

 

**Task:**

Write a Python script that checks if there is a new release of the Events dataset (the first link in the list). If there is a new dataset available, download the CSV file and save it as Parquet file. The resulting Parquet file should grow incrementally, i.e., it should be updated whenever there is a new release – which is usually every 15 minutes. One important requirement is, that the resulting Parquet file does not contain duplicates.


**Proposed Solution:**
1. Hit the URL and check if the release is new or still reflecting old. If its an older release then ignore. 

2. If its a newer release then dowload it into pandas dataframe. We enforce column names and datatypes as mentioned in official schema document mentioned in the source.

3. As per link mentioned in the sources, Same news can be reshared multiple times which ends up wt lot of duplicates

4. We refer to column "SOURCEURL" and use it to drop the duplicates within the current events release

5. Current events release might also have some duplicates with previous releases. To resolve this we need to check duplicates against entire parquet table, which could become troublesome if table size is huge.

6. Hence we always check for previous partitions data and compare it with current events data to remove duplicates. Hence for next release the previous partition with have no duplicates and so on. This helps to not scan entire table


7. After cleaning we make use of dask and pyarrow libraries to write it into parquet file. If parquet file is not present for first time it would be created else we append to the same file. 

In [1]:
#Import the required libraries
from urllib.request import urlopen
from bs4 import BeautifulSoup
import zipfile,io
import pandas as pd
import glob
import os
import sys
import schedule
import time


In [2]:
!python -m pip install 'fsspec>=0.3.3'
!pip install fastparquet
!pip install schedule

In [3]:
import pyarrow.parquet as pq
import pyarrow as pa
import dask.dataframe as dd 
import fastparquet

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
url = "http://data.gdeltproject.org/gdeltv2/lastupdate.txt"

In [6]:
# file_list = sorted(glob.glob("/content/drive/MyDrive/datasets/*.CSV"))
# file_list

In [7]:
#Enforce the schema of CSV file
dataframe_field_types = [
    ('GlobalEventID', 'int64'),
    ('Day', 'int64'),
    ('MonthYear', 'int64'),
    ('Year', 'int64'),
    ('FractionDate', 'float64'),
    ('Actor1Code', 'object'),
    ('Actor1Name', 'object'),
    ('Actor1CountryCode', 'object'),
    ('Actor1KnownGroupCode', 'object'),
    ('Actor1EthnicCode', 'object'),
    ('Actor1Religion1Code', 'object'),
    ('Actor1Religion2Code', 'object'),
    ('Actor1Type1Code', 'object'),
    ('Actor1Type2Code', 'object'),
    ('Actor1Type3Code', 'object'),
    ('Actor2Code', 'object'),
    ('Actor2Name', 'object'),
    ('Actor2CountryCode', 'object'),
    ('Actor2KnownGroupCode', 'object'),
    ('Actor2EthnicCode', 'object'),
    ('Actor2Religion1Code', 'object'),
    ('Actor2Religion2Code', 'object'),
    ('Actor2Type1Code', 'object'),
    ('Actor2Type2Code', 'object'),
    ('Actor2Type3Code', 'object'),
    ('IsRootEvent', 'int64'),
    ('EventCode', 'object'),
    ('EventBaseCode', 'object'),
    ('EventRootCode', 'object'),
    ('QuadClass', 'int64'),
    ('GoldsteinScale', 'float64'),
    ('NumMentions', 'int64'),
    ('NumSources', 'int64'),
    ('NumArticles', 'int64'),
    ('AvgTone', 'float64'),
    ('Actor1Geo_Type', 'int64'),
    ('Actor1Geo_Fullname', 'object'),
    ('Actor1Geo_CountryCode', 'object'),
    ('Actor1Geo_ADM1Code', 'object'),
    ('Actor1Geo_ADM2Code', 'object'),
    ('Actor1Geo_Lat', 'float64'),
    ('Actor1Geo_Long', 'float64'),
    ('Actor1Geo_FeatureID', 'object'),
    ('Actor2Geo_Type', 'int64'),
    ('Actor2Geo_Fullname', 'object'),
    ('Actor2Geo_CountryCode', 'object'),
    ('Actor2Geo_ADM1Code', 'object'),
    ('Actor2Geo_ADM2Code', 'object'),
    ('Actor2Geo_Lat', 'float64'),
    ('Actor2Geo_Long', 'float64'),
    ('Actor2Geo_FeatureID', 'object'),
    ('ActionGeo_Type', 'int64'),
    ('ActionGeo_Fullname', 'object'),
    ('ActionGeo_CountryCode', 'object'),
    ('ActionGeo_ADM1Code', 'object'),
    ('ActionGeo_ADM2Code', 'object'),
    ('ActionGeo_Lat', 'float64'),
    ('ActionGeo_Long', 'float64'),
    ('ActionGeo_FeatureID', 'object'),
    ('DATEADDED', 'int64'),
    ('SOURCEURL', 'object')
    ]

In [8]:
#Initilizing field types and column names for given data
dataframe_field_type_dict = {}
ind = 0
for item in dataframe_field_types:
  dataframe_field_type_dict[ind] = item[1]
  ind = ind+1

dataframe_field_name_lst = []

for item in dataframe_field_types:
  dataframe_field_name_lst.append(item[0])


In [9]:
#Enforce the schema of Parquet file
myfields = [
    ('GlobalEventID', pa.int64()),
    ('Day', pa.int64()),
    ('MonthYear', pa.int64()),
    ('Year', pa.int64()),
    ('FractionDate', pa.float64()),
    ('Actor1Code', pa.string()),
    ('Actor1Name', pa.string()),
    ('Actor1CountryCode', pa.string()),
    ('Actor1KnownGroupCode', pa.string()),
    ('Actor1EthnicCode', pa.string()),
    ('Actor1Religion1Code', pa.string()),
    ('Actor1Religion2Code', pa.string()),
    ('Actor1Type1Code', pa.string()),
    ('Actor1Type2Code', pa.string()),
    ('Actor1Type3Code', pa.string()),
    ('Actor2Code', pa.string()),
    ('Actor2Name', pa.string()),
    ('Actor2CountryCode', pa.string()),
    ('Actor2KnownGroupCode', pa.string()),
    ('Actor2EthnicCode', pa.string()),
    ('Actor2Religion1Code', pa.string()),
    ('Actor2Religion2Code', pa.string()),
    ('Actor2Type1Code', pa.string()),
    ('Actor2Type2Code', pa.string()),
    ('Actor2Type3Code', pa.string()),
    ('IsRootEvent', pa.int64()),
    ('EventCode', pa.string()),
    ('EventBaseCode', pa.string()),
    ('EventRootCode', pa.string()),
    ('QuadClass', pa.int64()),
    ('GoldsteinScale', pa.float64()),
    ('NumMentions', pa.int64()),
    ('NumSources', pa.int64()),
    ('NumArticles', pa.int64()),
    ('AvgTone', pa.float64()),
    ('Actor1Geo_Type', pa.int64()),
    ('Actor1Geo_Fullname', pa.string()),
    ('Actor1Geo_CountryCode', pa.string()),
    ('Actor1Geo_ADM1Code', pa.string()),
    ('Actor1Geo_ADM2Code', pa.string()),
    ('Actor1Geo_Lat', pa.float64()),
    ('Actor1Geo_Long', pa.float64()),
    ('Actor1Geo_FeatureID', pa.string()),
    ('Actor2Geo_Type', pa.int64()),
    ('Actor2Geo_Fullname', pa.string()),
    ('Actor2Geo_CountryCode', pa.string()),
    ('Actor2Geo_ADM1Code', pa.string()),
    ('Actor2Geo_ADM2Code', pa.string()),
    ('Actor2Geo_Lat', pa.float64()),
    ('Actor2Geo_Long', pa.float64()),
    ('Actor2Geo_FeatureID', pa.string()),
    ('ActionGeo_Type', pa.int64()),
    ('ActionGeo_Fullname', pa.string()),
    ('ActionGeo_CountryCode', pa.string()),
    ('ActionGeo_ADM1Code', pa.string()),
    ('ActionGeo_ADM2Code', pa.string()),
    ('ActionGeo_Lat', pa.float64()),
    ('ActionGeo_Long', pa.float64()),
    ('ActionGeo_FeatureID', pa.string()),
    ('DATEADDED', pa.int64()),
    ('SOURCEURL', pa.string()),
    ]

In [10]:
parquet_schema_dict = {}
for item in myfields:
  parquet_schema_dict[item[0]] = item[1]

In [11]:

gdelt_parquet_file_path = '/content/gdelt_events.parquet'

# Read from given link

* Function that  would hit the link, read the file
* Later this would be passed to another function to return the dataframe

In [12]:
# From given file path generate pandas dataframe with column names and data types as per schema document
def read_csv_to_dataframe(fil_path, field_dicts, separator, field_types):
  df = pd.read_csv(fil_path, header=None, dtype=field_dicts, sep=separator, names=field_types)
  return df


In [13]:
# Hit the url and down load the file
def load_new_data(url=url):
    
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")


    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)

    print(text)
    resp = urlopen(text[text.find("http"):text.find("zip")+3])
    z = zipfile.ZipFile(io.BytesIO(resp.read()))
    currflpth=z.open(z.namelist()[0])
    return read_csv_to_dataframe(currflpth,field_dicts=dataframe_field_type_dict,separator="\t", field_types=dataframe_field_name_lst)

In [14]:
current_mdf=load_new_data(url)

97368 94d0ee7a187dc36756849b8448a8893c http://data.gdeltproject.org/gdeltv2/20210603204500.export.CSV.zip
167216 ea70f65cb6149b8bc38d2022c23736cd http://data.gdeltproject.org/gdeltv2/20210603204500.mentions.CSV.zip
7446267 e789b3333692668e10f5aa67152587d0 http://data.gdeltproject.org/gdeltv2/20210603204500.gkg.csv.zip


In [15]:
current_mdf.shape

(1381, 61)

In [16]:
current_mdf.head()

,GlobalEventID,Day,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,Actor1Religion2Code,Actor1Type1Code,Actor1Type2Code,Actor1Type3Code,Actor2Code,Actor2Name,Actor2CountryCode,Actor2KnownGroupCode,Actor2EthnicCode,Actor2Religion1Code,Actor2Religion2Code,Actor2Type1Code,Actor2Type2Code,Actor2Type3Code,IsRootEvent,EventCode,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone,Actor1Geo_Type,Actor1Geo_Fullname,Actor1Geo_CountryCode,Actor1Geo_ADM1Code,Actor1Geo_ADM2Code,Actor1Geo_Lat,Actor1Geo_Long,Actor1Geo_FeatureID,Actor2Geo_Type,Actor2Geo_Fullname,Actor2Geo_CountryCode,Actor2Geo_ADM1Code,Actor2Geo_ADM2Code,Actor2Geo_Lat,Actor2Geo_Long,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_Fullname,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,988405706,20200603,202006,2020,2020.4192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ESPBUS,SPANISH,ESP,NaN,NaN,NaN,NaN,BUS,NaN,NaN,1,042,042,04,1,1.9,2,1,2,-0.819672,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Canada,CA,CA,NaN,60.0000,-96.0000,CA,1,Canada,CA,CA,NaN,60.0000,-96.0000,CA,20210603204500,http://www.journal-aviation.com/en/news/46098-...
1,988405707,20200603,202006,2020,2020.4192,BHR,BAHRAIN,BHR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IGOUNOHLHWHO,WORLD HEALTH ORGANIZATION,NaN,UNO,NaN,NaN,NaN,IGO,HLH,NaN,1,030,030,03,1,4.0,10,1,10,0.000000,1,Bahrain,BA,BA,NaN,26.0000,50.5000,BA,1,Bahrain,BA,BA,NaN,26.0000,50.5000,BA,1,Bahrain,BA,BA,NaN,26.0000,50.5000,BA,20210603204500,https://arynews.tv/en/bahrain-booster-sinophar...
2,988405708,20200603,202006,2020,2020.4192,CVL,HUNTER,NaN,NaN,NaN,NaN,NaN,CVL,NaN,NaN,HLH,SURGEON,NaN,NaN,NaN,NaN,NaN,HLH,NaN,NaN,0,010,010,01,1,0.0,1,1,1,-3.456221,3,"Winston-Salem, North Carolina, United States",US,USNC,NC067,36.0999,-80.2442,997498,3,"Winston-Salem, North Carolina, United States",US,USNC,NaN,36.0999,-80.2442,997498,3,"Winston-Salem, North Carolina, United States",US,USNC,NaN,36.0999,-80.2442,997498,20210603204500,https://www.carolinajournal.com/news-article/s...
3,988405709,20200603,202006,2020,2020.4192,CVL,HUNTER,NaN,NaN,NaN,NaN,NaN,CVL,NaN,NaN,HLH,SURGEON,NaN,NaN,NaN,NaN,NaN,HLH,NaN,NaN,0,110,110,11,3,-2.0,1,1,1,-3.456221,3,"Winston-Salem, North Carolina, United States",US,USNC,NC067,36.0999,-80.2442,997498,3,"Winston-Salem, North Carolina, United States",US,USNC,NaN,36.0999,-80.2442,997498,3,"Winston-Salem, North Carolina, United States",US,USNC,NaN,36.0999,-80.2442,997498,20210603204500,https://www.carolinajournal.com/news-article/s...
4,988405710,20200603,202006,2020,2020.4192,CVL,COMMUNITY,NaN,NaN,NaN,NaN,NaN,CVL,NaN,NaN,USA,DETROIT,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,051,051,05,1,3.4,1,1,1,4.369919,2,"Connecticut, United States",US,USCT,NaN,41.5834,-72.7622,CT,2,"Connecticut, United States",US,USCT,NaN,41.5834,-72.7622,CT,2,"Connecticut, United States",US,USCT,NaN,41.5834,-72.7622,CT,20210603204500,https://www.vcstar.com/story/news/2021/06/03/g...


# Write to Parquet files

* Functions that could read and write into aprquet file
* Main driver program that orchestrates all the flow

In [17]:
"""
The events data from current time will have duplicates
As per sources mentioned below, News information is usually duplicated.
Hence we use SOURCEURL as primary key and remove such duplicates from current events file
"""
def drop_duplicates_within_dataframe(tmpdf, primary_key="SOURCEURL"):
  tmpdf = tmpdf.drop_duplicates(subset=[primary_key], keep='last')
  return tmpdf;

"""
The current events dataframe may have some duplicates with previous data.
As some news information might get shared in both the files.
Hence we need to remove this from current events file by referening previous events data.

Reading entire parquet table and finding duplicates is a time consuming task especially if table sze is large.
Hence we only read previous partition of parquet file and assume all the oher partitions have uniqur data.
We compare this previous partition data with current events data.
Duplicates from current events data are dropped.
Hence this helps us to not read entire parquet table everytime and only read a single partition.
"""

def drop_duplicates_between_dataframes(previous_partition_dataframe, current_dataframe, primary_key = "SOURCEURL"):
  result = pd.merge(current_dataframe, previous_partition_dataframe[primary_key], on=[primary_key, primary_key], how="left", indicator=True).query('_merge == "left_only"').drop('_merge', 1)
  return result;


In [18]:
"""
Get the latest partition from parquet file.
We can get more previous partitions by setting part_num more than 1 
"""
def get_latest_partition_from_parquet(gdelt_parquet_file_path, part_num=1, engine = 'fastparquet'):
  dask_dataframe = dd.read_parquet(gdelt_parquet_file_path, engine =engine)
  latest_partition_dataframe = dask_dataframe.partitions[dask_dataframe.npartitions - part_num ]
  print(latest_partition_dataframe.index.count().compute())
  print(latest_partition_dataframe.head())
  df = latest_partition_dataframe.compute()
  return df;


# # dask_dataframe = dd.read_parquet(gdelt_parquet_file_path, engine = 'fastparquet')

# mydf = get_latest_partition_from_parquet(gdelt_parquet_file_path)
# mydf.head()

In [19]:
"""
1. Check if parquet file already exists. If it is not present then start new parquet file.
Convert pandas dataframe to dask dataframe.
Put it into a parquet table

2. Check if parquet file already exists. If it is present then append to parquet file.
Here we are not concerned about number partitions. But if we need to reduce number of small partitions then they ahev to be merged.
In APAche spark this can be accomplisged by Coalesce function. But since we have not used spark we dont address this problem for now.
Convert pandas dataframe to dask dataframe.
Append it into a parquet table
"""

def write_to_parquet(parquet_table_path, pandas_dataframe, chunksize=10000, primary_key = "GlobalEventID"):
  if not(os.path.exists(parquet_table_path)):
    print("Creating new parquet")
    pandas_dataframe.set_index([primary_key], inplace=True)
    dask_dataframe = dd.from_pandas(pandas_dataframe, chunksize=chunksize)
    dask_dataframe.to_parquet(path=parquet_table_path,engine = 'fastparquet' )
    print(dask_dataframe.head(3))
  
  else:
    print("Appending to Parquet")
    existing_parquet_dataframe = dd.read_parquet(path=parquet_table_path, engine="pyarrow")
    print("Number of partitions: {}".format(existing_parquet_dataframe.npartitions))
    most_recent_partition = existing_parquet_dataframe.npartitions

    pandas_dataframe.set_index([primary_key], inplace=True)
    dask_dataframe = dd.from_pandas(pandas_dataframe, chunksize=chunksize)
    dask_dataframe.to_parquet(path=parquet_table_path, engine = 'fastparquet', ignore_divisions=True, append=True)
    # print(dask_dataframe.head(5, npartitions=most_recent_partition-1))
  
  return

In [20]:
# Read csv file into a dataframe of column names, field types
def read_csv_to_dataframe(fil_path, field_dicts, separator, field_types):
  df = pd.read_csv(fil_path, header=None, dtype=field_dicts, sep=separator, names=field_types)
  return df


In [21]:
l,r = ['1',2]

In [22]:
"""
Main driver program
1. Gets the events file posted at GDELT
2. Remove duplicates both within file and across file.
3. Write the cleaned file into parquet
"""

def driver_program(url=url, gdelt_parquet_file_path=gdelt_parquet_file_path):
  events_df = load_new_data(url=url)
  print(events_df.shape)
  events_df = drop_duplicates_within_dataframe(events_df)
  print(events_df.shape)

  if (os.path.exists(gdelt_parquet_file_path)):
    latest_partition_dataframe = get_latest_partition_from_parquet(gdelt_parquet_file_path)
    print(latest_partition_dataframe.shape)
    events_df = drop_duplicates_between_dataframes(latest_partition_dataframe, events_df)
    print(events_df.shape)
  
  # Write to parquet file if we have non zero records from cleaned events dataframe.
  # If new events file has full duplicates then after cleaning its row count may be zero
  # Hence no need to write those files
  if (events_df.shape[0] > 0):
    write_to_parquet(gdelt_parquet_file_path, events_df)

  return


In [23]:
driver_program()

97368 94d0ee7a187dc36756849b8448a8893c http://data.gdeltproject.org/gdeltv2/20210603204500.export.CSV.zip
167216 ea70f65cb6149b8bc38d2022c23736cd http://data.gdeltproject.org/gdeltv2/20210603204500.mentions.CSV.zip
7446267 e789b3333692668e10f5aa67152587d0 http://data.gdeltproject.org/gdeltv2/20210603204500.gkg.csv.zip
(1381, 61)
(318, 61)
318
                    Day  ...                                          SOURCEURL
GlobalEventID            ...                                                   
988405707      20200603  ...  https://arynews.tv/en/bahrain-booster-sinophar...
988405710      20200603  ...  https://www.vcstar.com/story/news/2021/06/03/g...
988405727      20210504  ...  https://www.theguardian.com/education/2021/jun...
988405728      20210504  ...  https://www.vanityfair.com/news/2021/06/andrew...
988405731      20210504  ...  https://www.forbes.com/sites/joewalsh/2021/06/...

[5 rows x 60 columns]
(318, 60)
(0, 61)


# Scheduler

Job has been scheduled to run every 15 minutes

In [24]:
#Schedule the script for every 15 minutes 
schedule.every(15).seconds.do(driver_program)

while True:
    schedule.run_pending()
    time.sleep(1)

97368 94d0ee7a187dc36756849b8448a8893c http://data.gdeltproject.org/gdeltv2/20210603204500.export.CSV.zip
167216 ea70f65cb6149b8bc38d2022c23736cd http://data.gdeltproject.org/gdeltv2/20210603204500.mentions.CSV.zip
7446267 e789b3333692668e10f5aa67152587d0 http://data.gdeltproject.org/gdeltv2/20210603204500.gkg.csv.zip
(1381, 61)
(318, 61)
318
                    Day  ...                                          SOURCEURL
GlobalEventID            ...                                                   
988405707      20200603  ...  https://arynews.tv/en/bahrain-booster-sinophar...
988405710      20200603  ...  https://www.vcstar.com/story/news/2021/06/03/g...
988405727      20210504  ...  https://www.theguardian.com/education/2021/jun...
988405728      20210504  ...  https://www.vanityfair.com/news/2021/06/andrew...
988405731      20210504  ...  https://www.forbes.com/sites/joewalsh/2021/06/...

[5 rows x 60 columns]
(318, 60)
(0, 61)
97368 94d0ee7a187dc36756849b8448a8893c http://data.gde

KeyboardInterrupt: ignored